<a href="https://colab.research.google.com/github/selfproclaimedgenius1706/Alzheimer-s-Disease-Detection/blob/main/distilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch

In [ ]:
%%capture
!python3 -m venv venv
!source venv/bin/activate
!pip install tensorflow transformers


In [ ]:
import tensorflow as tf
from tensorflow.keras import activations, optimizers, losses

#Maybe I should not use the TFDistillBertForSequenceClassification
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

import pickle


# TOKENIZATION !!!

In [ ]:
df = pd.read_csv('dataToPadding.csv')
df = df[~df['Seri'].isin(['s004','s007','s024', 's096','s084','s085'])]

In [ ]:
#Tokenization !!!
def construct_encodings(data, tokenizer, max_len, truncation=True, padding=True):
    encodings = tokenizer(data, max_length=max_len, truncation=truncation, padding=padding)

    #Alert if any sample > max len
    for idx, input_ids in enumerate(encodings['input_ids']):
        if len(input_ids) > max_len:
            print(f"Warning: Sequence at index {idx} is longer than {max_len} tokens.")

    return encodings



In [ ]:
#Chia x(feature),y(label)
x = df['PreprocessedContent'].tolist()
y  = df['Label'].tolist()

#load model
MODEL_NAME = 'distilbert-base-uncased'
tkzr = DistilBertTokenizer.from_pretrained(MODEL_NAME)

#Tokenize
encodings = construct_encodings(x, tokenizer = tkzr, max_len=40)


In [ ]:
# Access the input_ids and attention_mask (để check thử coi shape tụi nó ra sao)
for i in range(20):
  input_ids = encodings['input_ids'][i]
  attention_mask = encodings['attention_mask'][i]
  input_ids = tf.convert_to_tensor(input_ids)
  attention_mask = tf.convert_to_tensor(attention_mask)

  print("Input IDs:", input_ids)
  print("Attention Mask:", attention_mask)
  print("="*50)

Input IDs: tf.Tensor(
[  101  2092  2045  2015  1037  2388  3061  2045  7910  7910 12699  1996
 10447  1998  7910  1996  3645  2330  1998  2648  1996  3332  2045  2015
  1037  3328  2007  1037  1039  9203  3328  1998  2017  2064  2156  2178
  7910  7910  2311   102], shape=(40,), dtype=int32)
Attention Mask: tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1], shape=(40,), dtype=int32)
Input IDs: tf.Tensor(
[  101  8307  2015  2893 16324  2041 11253  1996 17387 15723  3061  2006
  1037  1996 14708  2015  6069  5955  2058  1998  1996  3057  3038 14021
  2232  2123  2102  2292  8307  2963  1998  1996 10756 17462 10447  1996
  5380  2770  2041   102], shape=(40,), dtype=int32)
Attention Mask: tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1], shape=(40,), dtype=int32)
Input IDs: tf.Tensor(
[  101  3100  2045  2015  1037  2210  2879  1998  2002  2015  2893  2002
  2015  3061  2006  1037  1998  2002  2015 

In [ ]:
#Ghép x và y lại dataset để train/test
def construct_tfdataset(encodings, y=None):
    if y:
        return tf.data.Dataset.from_tensor_slices((dict(encodings),y))
    else:
        # this case is used when making predictions on unseen samples after training
        return tf.data.Dataset.from_tensor_slices(dict(encodings))

tfdataset = construct_tfdataset(encodings, y)


## APPROACH 1 : USING HIGH-LEVEL API TO TRAIN + SPLIT 7-3 (TRAIN-TEST)

In [ ]:
#70% for train, 30% for test
TEST_SPLIT = 0.3
train_size = int(len(x) * (1-TEST_SPLIT))

#Set batch size
BATCH_SIZE = 2

#shuffle lên
tfdataset = tfdataset.shuffle(len(x))

tfdataset_train = tfdataset.take(train_size)
tfdataset_train = tfdataset_train.batch(BATCH_SIZE)

tfdataset_test = tfdataset.skip(train_size)
tfdataset_test = tfdataset_test.batch(BATCH_SIZE)

In [ ]:
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import Adam
import numpy as np


In [ ]:
N_EPOCHS = 20
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
optimizer = optimizers.Adam(learning_rate=1e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
model.fit(tfdataset_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/20
36/36 [==============================] - 71s 1s/step - loss: 0.6902 - accuracy: 0.5556
Epoch 2/20
36/36 [==============================] - 51s 1s/step - loss: 0.6287 - accuracy: 0.7222
Epoch 3/20
36/36 [==============================] - 50s 1s/step - loss: 0.5613 - accuracy: 0.7083
Epoch 4/20
36/36 [==============================] - 51s 1s/step - loss: 0.4196 - accuracy: 0.8472
Epoch 5/20
36/36 [==============================] - 51s 1s/step - loss: 0.2793 - accuracy: 0.9444
Epoch 6/20
36/36 [==============================] - 52s 1s/step - loss: 0.1972 - accuracy: 0.9444
Epoch 7/20
36/36 [==============================] - 52s 1s/step - loss: 0.1407 - accuracy: 0.9583
Epoch 8/20
36/36 [==============================] - 52s 1s/step - loss: 0.1201 - accuracy: 0.9861
Epoch 9/20
36/36 [==============================] - 52s 1s/step - loss: 0.0777 - accuracy: 0.9861
Epoch 10/20
36/36 [==============================] - 51s 1s/step - loss: 0.0373 - accuracy: 1.0000
Epoch 11/20
36/36 [

In [ ]:
#Check accuracy
benchmarks = model.evaluate(tfdataset_test, return_dict=True, batch_size=BATCH_SIZE)
accuracy = benchmarks['accuracy']
print(f'The accuracy of the model is {round(accuracy*100)}%')

16/16 [==============================] - 4s 159ms/step - loss: 0.0045 - accuracy: 1.0000
The accuracy of the model is 100%


# TEST ON NEW DATASET

In [ ]:
testDF = pd.read_csv('preprocessed_data2020.csv')
test_text = testDF['Preprocessed_Content']
test_label = testDF['label']

# Print the shape of the extracted arrays
print("test_text shape:", test_text.shape)
print("test_label shape:", test_label.shape)

tokens_test = tkzr.batch_encode_plus(
    test_text.tolist(),
    max_length = 40,
    pad_to_max_length=True,
    truncation=True
)

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_label.tolist())

test_text shape: (48,)
test_label shape: (48,)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from sklearn.metrics import classification_report
import tensorflow as tf

# Convert PyTorch tensors to TensorFlow tensors
test_seq_tf = tf.convert_to_tensor(test_seq.numpy())
test_mask_tf = tf.convert_to_tensor(test_mask.numpy())

# Use tf.config.list_physical_devices to check for GPU availability
gpu_available = tf.config.list_physical_devices('GPU')

# Use GPU if available, otherwise use CPU
device = "/GPU:0" if gpu_available else "/CPU:0"

with tf.device(device):
    preds = model([test_seq_tf, test_mask_tf])

# Access the logits and convert them to a NumPy array
logits = preds.logits.numpy()
preds = np.argmax(logits, axis=1)
print(classification_report(test_y, preds))


              precision    recall  f1-score   support

           0       0.78      0.75      0.77        24
           1       0.76      0.79      0.78        24

    accuracy                           0.77        48
   macro avg       0.77      0.77      0.77        48
weighted avg       0.77      0.77      0.77        48



# SAVE THE MODEL

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


model_directory = "/content/drive/MyDrive/DistilBERT"
model.save_pretrained(model_directory)


Mounted at /content/drive


##APPROACH 2 : TRAINING LOOP + USING K-FOLD CROSS VALIDATION

In [ ]:
# Initialize KFold cross-validator
N_FOLDS = 5
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
BATCH_SIZE = 2
N_EPOCHS = 10

# Initialize lists to store metrics across folds
fold_train_losses = []
fold_train_accuracies = []
fold_val_losses = []
fold_val_accuracies = []

# Convert x and y to NumPy arrays
x = np.array(encodings['input_ids'])
y = np.array(y)

#TRAINING LOOP
for fold, (train_idx, val_idx) in enumerate(kf.split(x)):
    print(f"Fold {fold + 1}/{N_FOLDS}")

    x_train, y_train = x[train_idx], y[train_idx]
    x_val, y_val = x[val_idx], y[val_idx]

    # Create and compile the model
    model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
    optimizer = Adam(learning_rate=1e-5)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy' , metrics=['accuracy'])

    # Create TensorFlow Datasets
    tfdataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    tfdataset_train = tfdataset_train.shuffle(len(x_train)).batch(BATCH_SIZE)

    tfdataset_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    tfdataset_val = tfdataset_val.batch(BATCH_SIZE)

    # Build the optimizer with the list of trainable variables
    optimizer.build(model.trainable_variables)

    # Train the model
    history = model.fit(tfdataset_train, epochs=N_EPOCHS, validation_data=tfdataset_val)

    # Store metrics for this fold
    fold_train_losses.append(history.history['loss'])
    fold_train_accuracies.append(history.history['accuracy'])
    fold_val_losses.append(history.history['val_loss'])
    fold_val_accuracies.append(history.history['val_accuracy'])

# Calculate average metrics across all folds
avg_train_loss = np.mean(fold_train_losses, axis=0)
avg_train_accuracy = np.mean(fold_train_accuracies, axis=0)
avg_val_loss = np.mean(fold_val_losses, axis=0)
avg_val_accuracy = np.mean(fold_val_accuracies, axis=0)

print("Average Training Loss:", avg_train_loss)
print("Average Training Accuracy:", avg_train_accuracy)
print("Average Validation Loss:", avg_val_loss)
print("Average Validation Accuracy:", avg_val_accuracy)



Fold 1/5


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/10
41/41 [==============================] - 84s 2s/step - loss: 1.0956 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 2/10
41/41 [==============================] - 68s 2s/step - loss: 0.6931 - accuracy: 0.5854 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 3/10
41/41 [==============================] - 62s 2s/step - loss: 0.6931 - accuracy: 0.4634 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 4/10
41/41 [==============================] - 64s 2s/step - loss: 0.6931 - accuracy: 0.5610 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 5/10
41/41 [==============================] - 64s 2s/step - loss: 0.6931 - accuracy: 0.6220 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 6/10
41/41 [==============================] - 63s 2s/step - loss: 0.6931 - accuracy: 0.5244 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 7/10
41/41 [==============================] - 63s 2s/step - loss: 0.6931 - accuracy: 0.4390 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 8/10
41/41 [==

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/10
41/41 [==============================] - 83s 2s/step - loss: 2.1844 - accuracy: 0.5122 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 2/10
41/41 [==============================] - 64s 2s/step - loss: 0.6931 - accuracy: 0.5854 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 3/10
41/41 [==============================] - 66s 2s/step - loss: 0.6931 - accuracy: 0.6585 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 4/10
41/41 [==============================] - 66s 2s/step - loss: 0.6931 - accuracy: 0.6463 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 5/10
41/41 [==============================] - 64s 2s/step - loss: 0.6931 - accuracy: 0.5854 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 6/10
41/41 [==============================] - 64s 2s/step - loss: 0.6931 - accuracy: 0.6220 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 7/10
41/41 [==============================] - 63s 2s/step - loss: 0.6931 - accuracy: 0.6098 - val_loss: 0.6931 - val_accuracy: 0.7143
Epoch 8/10
41/41 [==

...

Epoch 1/10
41/41 [==============================] - 86s 2s/step - loss: 0.8115 - accuracy: 0.5610 - val_loss: 0.6931 - val_accuracy: 0.8095
Epoch 2/10
41/41 [==============================] - 65s 2s/step - loss: 0.8169 - accuracy: 0.4024 - val_loss: 0.6931 - val_accuracy: 0.2857
Epoch 3/10
41/41 [==============================] - 65s 2s/step - loss: 0.8482 - accuracy: 0.5610 - val_loss: 0.6931 - val_accuracy: 0.2857
Epoch 4/10
41/41 [==============================] - 63s 2s/step - loss: 0.6931 - accuracy: 0.5488 - val_loss: 0.6931 - val_accuracy: 0.2857
Epoch 5/10
41/41 [==============================] - 64s 2s/step - loss: 0.6931 - accuracy: 0.5610 - val_loss: 0.6931 - val_accuracy: 0.2857
Epoch 6/10
41/41 [==============================] - 63s 2s/step - loss: 0.6931 - accuracy: 0.5732 - val_loss: 0.6931 - val_accuracy: 0.2857
Epoch 7/10
41/41 [==============================] - 63s 2s/step - loss: 0.6931 - accuracy: 0.5488 - val_loss: 0.6931 - val_accuracy: 0.2857
Epoch 8/10
41/41 [==

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/10
42/42 [==============================] - 90s 2s/step - loss: 1.4080 - accuracy: 0.5301 - val_loss: 0.6931 - val_accuracy: 0.3000
Epoch 2/10
42/42 [==============================] - 68s 2s/step - loss: 0.8060 - accuracy: 0.5301 - val_loss: 0.6931 - val_accuracy: 0.7500
Epoch 3/10
42/42 [==============================] - 68s 2s/step - loss: 0.6931 - accuracy: 0.3614 - val_loss: 0.6931 - val_accuracy: 0.7500
Epoch 4/10
42/42 [==============================] - 68s 2s/step - loss: 0.6931 - accuracy: 0.5301 - val_loss: 0.6931 - val_accuracy: 0.7500
Epoch 5/10
42/42 [==============================] - 71s 2s/step - loss: 0.6931 - accuracy: 0.4458 - val_loss: 0.6931 - val_accuracy: 0.7500
Epoch 6/10
42/42 [==============================] - 67s 2s/step - loss: 0.6931 - accuracy: 0.5422 - val_loss: 0.6931 - val_accuracy: 0.7500
Epoch 7/10
42/42 [==============================] - 68s 2s/step - loss: 0.6931 - accuracy: 0.5542 - val_loss: 0.6931 - val_accuracy: 0.7500
Epoch 8/10
42/42 [==

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/10
42/42 [==============================] - 100s 2s/step - loss: 1.4828 - accuracy: 0.4699 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 2/10
42/42 [==============================] - 67s 2s/step - loss: 0.6931 - accuracy: 0.4940 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 3/10
42/42 [==============================] - 67s 2s/step - loss: 0.6931 - accuracy: 0.5301 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 4/10
42/42 [==============================] - 67s 2s/step - loss: 0.6931 - accuracy: 0.4699 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 5/10
42/42 [==============================] - 66s 2s/step - loss: 0.6931 - accuracy: 0.4699 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 6/10
42/42 [==============================] - 66s 2s/step - loss: 0.6931 - accuracy: 0.4940 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 7/10
42/42 [==============================] - 66s 2s/step - loss: 0.6931 - accuracy: 0.5181 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 8/10
42/42 [=

In [ ]:
testDF = pd.read_csv('preprocessed_data2020.csv')
test_text = testDF['Preprocessed_Content']
test_label = testDF['label']

# Print the shape of the extracted arrays
print("test_text shape:", test_text.shape)
print("test_label shape:", test_label.shape)

tokens_test = tkzr.batch_encode_plus(
    test_text.tolist(),
    max_length = 40,
    pad_to_max_length=True,
    truncation=True
)

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_label.tolist())

test_text shape: (48,)
test_label shape: (48,)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from sklearn.metrics import classification_report
import tensorflow as tf

# Convert PyTorch tensors to TensorFlow tensors
test_seq_tf = tf.convert_to_tensor(test_seq.numpy())
test_mask_tf = tf.convert_to_tensor(test_mask.numpy())

# Use tf.config.list_physical_devices to check for GPU availability
gpu_available = tf.config.list_physical_devices('GPU')

# Use GPU if available, otherwise use CPU
device = "/GPU:0" if gpu_available else "/CPU:0"

with tf.device(device):
    preds = model([test_seq_tf, test_mask_tf])

# Access the logits and convert them to a NumPy array
logits = preds.logits.numpy()
preds = np.argmax(logits, axis=1)
print(classification_report(test_y, preds))


              precision    recall  f1-score   support

           0       1.00      0.04      0.08        24
           1       0.51      1.00      0.68        24

    accuracy                           0.52        48
   macro avg       0.76      0.52      0.38        48
weighted avg       0.76      0.52      0.38        48



## FUNCTIONS ĐỂ TEST ON REAL DATA (COMPLETELY NEW DATA)

In [ ]:
#Hàm sài để test 1 sample data lẻ (1 transcript)
def create_predictor(model, model_name, max_len):
  tkzr = DistilBertTokenizer.from_pretrained(model_name)
  def predict_proba(text):
      x = [text]

      encodings = construct_encodings(x, tkzr, max_len=max_len)
      tfdataset = construct_tfdataset(encodings)
      tfdataset = tfdataset.batch(1)

      preds = model.predict(tfdataset).logits
      preds = activations.softmax(tf.convert_to_tensor(preds)).numpy()
      return preds[0][0]

  return predict_proba

In [ ]:
#VÍ DỤ CÁCH SÀI

clf = create_predictor(model, MODEL_NAME,20)
proba = clf('''Hello
''')
if proba>0.5:
  print(f'Model prediction : Alzheimer')
  print(f'The percentage of Alzheimer is : {round(proba*100)}%')
else:
  print("Model prediction : Healthy")
  print(f'The percentage of healthy is : {round(100-proba*100)}%')


In [ ]:
#Hàm sài để test 1 dataset luôn (nhiều transcript cùng lúc)
def create_predictor(model, model_name, max_len):
    tkzr = DistilBertTokenizer.from_pretrained(model_name)

    def predict_proba(text):
        x = [text]

        encodings = construct_encodings(x, tkzr, max_len=max_len)
        tfdataset = construct_tfdataset(encodings)
        tfdataset = tfdataset.batch(1)

        preds = model.predict(tfdataset).logits
        preds = activations.softmax(tf.convert_to_tensor(preds)).numpy()
        return preds[0][0]

    return predict_proba


In [ ]:
#VÍ DỤ CÁCH SÀI

# Load the new DataFrame
new_df = pd.read_csv("./preprocessed_data2020.csv")

# Define the maximum sequence length
max_len = 128  # Adjust this value as needed

# Create the predictor function
predict_proba = create_predictor(model, "distilbert-base-uncased", 50)

# Apply the predictor function to each row of the new DataFrame
new_df["Predictions"] = new_df["Preprocessed_Content"].apply(predict_proba)

csv_filename = "test_data2020(1).csv"
new_df.to_csv(csv_filename, index=False)

from google.colab import files
files.download(csv_filename)

In [ ]:
Dear Admission Officer,

I am Huong Thi Thanh Ha, a Lecturer at International University, Vietnam National University, Ho Chi Minh City. I earned my PhD in Neuroscience at Stanford University in 2018 and returned to Vietnam after graduation to establish the Brain Health Lab, which aims to address mental health issues and cognitive-aging disorders in Vietnam. My partner, Dr. Lua Thi Ngo, and I are currently mentoring a team of 4 high school students from VNU HCM - High School for the Gifted in research related to using Natural Language Processing to detect Alzheimer’s Disease based on language impairment through speech. Xuan Khoi is the leader of the team. It is my pleasure to recommend Khoi for the Computer Science program at your institution.

During the research period, Khoi consistently demonstrated exceptional leadership as he guided his team through the research process. His main responsibilities included reviewing research papers to define a research direction for the team, assigning tasks to each team member, and monitoring their weekly progress. One of the biggest challenges they encountered was the low and fluctuating accuracy of the model after the training process. During that time, Khoi played a pivotal role in motivating his team and keeping their spirits high. He dedicated a significant amount of time to immersing himself in various research papers to find a new technique to improve the model's accuracy. Initially, he attempted a fine-tuning approach, which failed to yield the desired results. After a week of persistence, Khoi discovered a viable solution: implementing a learning rate scheduler. I am truly impressed by Khoi's patience, resilience, and his ability to boost the team's morale during the most challenging times.

Regarding work ethics, I observed that Khoi was not only diligent in his work but also detail-oriented in the way he updated both me and Ms. Lua about the team's progress every week. His presentations clearly demonstrated his contributions to the research and his remarkable ability to provide thorough responses to all of mine and Ms. Lua's questions. This exemplified his comprehensive understanding not only of his own work but also of the team's weekly processes as a whole. In the presentations, he would also share his findings on effective training techniques, including the Learning Rate Scheduler, Grid Search, and 10-fold Cross-Validation.

In closing, I have no doubt that Khoi possesses the qualities, skills, and dedication required to excel in intensive Computer Science research. His leadership, combined with his infectious passion for the field of Artificial Intelligence and his remarkable work ethic, undoubtedly make him an outstanding candidate for your institution.

Sincerely,
